# Looping Through Numpy Arrays and Selecting Maximum Value

This notebook loops through multiple numpy NDVI arrays and outputs a single array with the maximum value for each element from all the arrays.  

In [389]:
# Read in libraries
import numpy as np
import rasterio
import os

### Loop

In [391]:
# Create a list of your files
lst = os.listdir('../data/NDVI/Copernicus_10m/clipped_and_max/ind_dates/') # your directory path

# Calculate the number of files in your directory
number_files = len(lst)

# Print
print("The number of files in the directory: ", number_files)
print("First three files in the directory: ", lst[:3])

The number of files in the directory:  28
First three files in the directory:  ['03Oct21_UYB.tif', '03Oct21_UYC.tif', '04Dec21_UYB.tif']


In [393]:
# Create an empty numpy array with the same shape as the others
outcome = np.empty((2846, 1760))

# Create a forloop which loops through every file, and selects the maximum value for each element in the array
for i in range(number_files):
    
    rastA = rasterio.open('../data/NDVI/Copernicus_10m/clipped_and_max/ind_dates/' + lst[i])
    rastA = rastA.read(1)*0.0001  # Multiply by 0.0001 as Python has not scaled the values when opening
    rastA = np.where(rastA != -3.2768, rastA, np.nan) # Convert values of -3.2768 to NA
    
    # Discarding NA values, find the maximum value for each element between the two arrays - outcome and rastA 
    total_max  = np.fmax(outcome, rastA)

    # if there are no values in rastA which are bigger than what has already been recorded - pass
    if (outcome == total_max).all() == True:
        pass
    # if there are values which are greater than that in "outcome", update "outcome"
    else:
        outcome = total_max

### Save Output

This file had a strange format where the 'dtype' is an integer, so decimal values do not appear. Therefore, we need to change the dtype to 'float'

In [398]:
# Copy the meta of one of the initial rasters
rast_ref = rasterio.open("../data/NDVI/Copernicus_10m/clipped_and_max/ind_dates/03Oct21_UYB.tif")
meta_x = rast_ref.meta.copy()

In [400]:
# Update the dtype to float
meta_x.update({"dtype":"float32"})

In [402]:
# Preview meta
meta_x

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -32768.0,
 'width': 1760,
 'height': 2846,
 'count': 1,
 'crs': CRS.from_epsg(32630),
 'transform': Affine(10.0, 0.0, 702560.0,
        0.0, -10.0, 5714770.0)}

In [404]:
# # Write raster layer as new raster file

# # Copy the profile of original raster layer (e.g. CRS, extent, etc.)
# profile_ref = rast1.profile

# # Write the file
# with rasterio.open("../data/NDVI/Copernicus_10m/clipped_and_max/max_all_ind_dates.tif", 'w', **meta_x) as dst:
#      dst.write(outcome, 1) # name of you variable (numpy array), and the band (band 1)

---

## Additional Code

### How to repeat process, but calling variables within the forloop

In [410]:
# Create test numpy arrays
np0 = np.array([0, 0, 6, 6])
np1 = np.array([0, 1, 6, 6])
np2 = np.array([0, 0, 0, 0])

In [412]:
# Creat an empty numpy array
outcome = np.empty((1, 4))

# Repeat forloop but callinng local variables rather than files from a directory
for i in range(3):

    rastA = globals()[f"np{i}"]                               # Use "globals()" to call variables within notebook
    rastA = np.where(rastA != -3.2768, rastA, np.nan)
    
    total_max  = np.fmax(outcome, rastA)

    if (outcome == total_max).all() == True:
        pass
    else:
        outcome = total_max

In [414]:
outcome

array([[0., 1., 6., 6.]])